# Bank Churners
**ML Project 1 - Spring 2022**

Collaborators: Ben DeSollar and Matt McDonnell

Descriptions: The first part should be a description which models/classifiers/regressors achieved the best 
    results with your datasets datasets, and what you did to improve those results along the way.

    - Through research we found that a called RandomForestClassifier worked the best after testing a couple different model including
    Niave Bayes and DecisionTreeClassifier. We improved our results by modifying our feature selection to include as many features as we 
    could to help predict the best values. Things like gender and Total relationship count did not seem as impptant as education level or
    credit limit. By modifying our featuiues during testing we were able to find the best model predictions.



### Step 1
Import the necessary packages and frameworks.

In [ ]:
import numpy as np
from mlwpy import *
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn import (datasets, neighbors,
                     naive_bayes,
                     model_selection as skms,
                     linear_model, dummy,
                     metrics,
                     pipeline,
                     preprocessing as skpre) 
import csv
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from seaborn import load_dataset
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.one_hot import OneHotEncoder
import category_encoders as ce

### Step 2
Read in .csv files and save as DataFrames.

In [ ]:
data_train_df = pd.read_csv("cars_class_train.csv")#, nrows=86108) 
data_test_df = pd.read_csv("cars_class_test.csv")#, nrows=86108)
data_train_ft = data_train_df.drop('type', axis=1)
data_train_tgt = data_train_df["type"]

### Step 3
Work with only the necessary features. We tried to choose features that were only about the users finances and not cateogies like gender or relationship count.
Yet things like marital status and education level seemed to have an impact on the prediction scores. 

In [ ]:
# To see what kind of data we're working with
print(data_test_df.dtypes)
print(data_train_ft.dtypes)
# print(data_train_ft['manufacturer'].unique())
# print(data_train_ft.head(5))

id                int64
url              object
region           object
region_url       object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
paint_color      object
image_url        object
description      object
state            object
lat             float64
long            float64
posting_date     object
dtype: object
Unnamed: 0        int64
url              object
region           object
region_url       object
price             int64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
pa

### Step 4
Work with a practice estimate (median age). Convert features to numerical values as necessary. 

For now, let's use a simple approach to estimate the age (and consider revisiting this estimate later) when it is missing by using the median from the other samples.

We also replaced any missing data in the dataset.

In [ ]:

# 5 6 8 12 13

features = ['url',
           'region',
           'region_url',
           'price',
           'year',
           'manufacturer',
           'model',
           'condition',
           'cylinders',
           'fuel',
           'odometer',
           'title_status',
           'transmission',
           'VIN',
           'drive',
           'size',
           'paint_color',
           'image_url',
           'description',
           'state',
           'lat',
           'long',
           'posting_date'
          ]
data_train_ft['posting_date'] = pd.to_datetime(data_train_ft['posting_date'], utc=True)
data_train_ft['posting_date'] = pd.to_datetime(data_train_ft['posting_date'])
data_train_ft['posting_date'] = (pd.DatetimeIndex(data_train_ft['posting_date']).day) + (pd.DatetimeIndex(data_train_ft['posting_date']).month*12) + ((2022 - pd.DatetimeIndex(data_train_ft['posting_date']).year)*365)
data_train_ft['posting_date'] = data_train_ft['posting_date'].astype(float)
print(data_train_ft['posting_date'].head(5))
data_test_df['posting_date'] = pd.to_datetime(data_test_df['posting_date'], utc=True)
data_test_df['posting_date'] = pd.to_datetime(data_test_df['posting_date'])
data_test_df['posting_date'] = (pd.DatetimeIndex(data_test_df['posting_date']).day) + (pd.DatetimeIndex(data_test_df['posting_date']).month*12) + ((2022 - pd.DatetimeIndex(data_test_df['posting_date']).year)*365)
data_test_df['posting_date'] = data_test_df['posting_date'].astype(float)
print(data_test_df['posting_date'].head(5))
data_train_ft = data_train_ft[features]
data_train_ft = data_train_ft.fillna(0)

data_test_ft = data_test_df
data_test_df = data_test_df[features]
data_test_df = data_test_df.fillna(0)

#encoder = ce.BackwardDifferenceEncoder(cols=features)
#encoder.fit_transform(data_train_ft, verbose=1).head()

#TE_encoder = TargetEncoder()
#data_train_ft = TE_encoder.fit_transform(data_train_ft, data_train_tgt)
#data_test_df = TE_encoder.transform(data_test_df)


#LOOE_encoder = LeaveOneOutEncoder()
#data_train_ft = LOOE_encoder.fit_transform(data_train_ft, data_train_tgt)
#data_test_df = LOOE_encoder.transform(data_test_df)


#LE_encoder = OrdinalEncoder(features)
#data_train_ft = LE_encoder.fit_transform(data_train_ft)
#data_test_df = LE_encoder.transform(data_test_df)

encoder = ce.BinaryEncoder(return_df=True)
data_train_ft = encoder.fit_transform(data_train_ft, data_train_tgt)
data_test_df = encoder.transform(data_test_df)

data_train_ft.head()
data_test_df.head()
'''
ftrsel = ftr_sel.SelectKBest(ftr_sel.f_classif, k=5)
ftrsel.fit_transform(data_train_ft, data_train_tgt)

keepers_idx = ftrsel.get_support()
print(keepers_idx)
# use target encoding to encode two categorical features

ftrsel = ftr_sel.SelectFromModel(ensemble.RandomForestClassifier(), 
                                 threshold='mean') # default
'''
ftrsel = ftr_sel.SelectKBest(ftr_sel.f_classif, 
                                  k=75)

ftrsel.fit_transform(data_train_ft, data_train_tgt)

print(np.array(data_train_ft.columns.value_counts().index)[ftrsel.get_support()])
print(data_train_ft.columns.value_counts().count())
#ftrsel = ftr_sel.RFE(ensemble.RandomForestClassifier(),
                    # n_features_to_select=7)
new_features = np.array(data_train_ft.columns.value_counts().index)[ftrsel.get_support()]
data_train_ft = data_train_ft[new_features]
data_test_df = data_test_df[new_features]

0               880.0000
1               892.0000
2               876.0000
3               887.0000
4               884.0000
Name: posting_date, dtype: float64
0               883.0000
1               891.0000
2               883.0000
3               884.0000
4               876.0000
Name: posting_date, dtype: float64
['region_0' 'title_status_0' 'VIN_3' 'VIN_2' 'VIN_1' 'VIN_0'
 'transmission_2' 'transmission_0' 'title_status_2' 'VIN_14' 'VIN_15'
 'VIN_16' 'drive_0' 'long' 'lat' 'state_5' 'state_4' 'state_3' 'state_2'
 'state_1' 'state_0' 'paint_color_3' 'paint_color_2' 'paint_color_1'
 'size_2' 'size_1' 'size_0' 'drive_2' 'drive_1' 'title_status_1' 'model_1'
 'model_0' 'manufacturer_5' 'manufacturer_3' 'year' 'fuel_2' 'price'
 'region_8' 'region_7' 'region_6' 'region_5' 'region_4' 'region_3'
 'region_2' 'model_3' 'model_4' 'model_5' 'model_6' 'fuel_1' 'fuel_0'
 'cylinders_3' 'cylinders_2' 'cylinders_1' 'cylinders_0' 'condition_2'
 'condition_1' 'condition_0' 'model_9' 'model_8' 'model

Here is an updated table after sorting through these features.

In [ ]:
print("data_train_ft:")
display(data_train_ft.head(10))
print("data_train_ft.info():")
display(data_train_ft.info())

data_train_ft = data_train_ft.astype(float)
data_test_df = data_test_df.astype(float)

print("data_train_ft:")
display(data_train_ft.head(10))
print("data_train_ft.info():")
display(data_train_ft.info())

# Analysis of the features looks like Education level, Credit Limit, and Income Level may be useful. 
# Features will need to be transformed to convert all into numbers to be analyzed.

data_train_ft:


,region_0,title_status_0,VIN_3,VIN_2,VIN_1,VIN_0,transmission_2,transmission_0,title_status_2,VIN_14,...,cylinders_3,cylinders_2,cylinders_1,cylinders_0,condition_2,condition_1,condition_0,model_9,model_8,model_7
0,0,0,0,0,0,0,1,0,1,0,...,1,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,0,1,0,...,1,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,1,0,1,0,...,0,1,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,1,0,...,1,0,0,0,1,1,0,0,0,0
4,0,0,0,0,0,0,0,0,1,1,...,1,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,1,0,1,1,...,0,1,0,0,1,0,1,0,0,0
6,0,0,0,0,0,0,1,0,1,1,...,0,1,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,1,0,1,1,...,1,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,1,0,0,0,0
9,0,0,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,1,0,0,0


data_train_ft.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200916 entries, 0 to 200915
Data columns (total 60 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   region_0        200916 non-null  int64  
 1   title_status_0  200916 non-null  int64  
 2   VIN_3           200916 non-null  int64  
 3   VIN_2           200916 non-null  int64  
 4   VIN_1           200916 non-null  int64  
 5   VIN_0           200916 non-null  int64  
 6   transmission_2  200916 non-null  int64  
 7   transmission_0  200916 non-null  int64  
 8   title_status_2  200916 non-null  int64  
 9   VIN_14          200916 non-null  int64  
 10  VIN_15          200916 non-null  int64  
 11  VIN_16          200916 non-null  int64  
 12  drive_0         200916 non-null  int64  
 13  long            200916 non-null  float64
 14  lat             200916 non-null  float64
 15  state_5         200916 non-null  int64  
 16  state_4         200916 non-null  i

None

data_train_ft:


,region_0,title_status_0,VIN_3,VIN_2,VIN_1,VIN_0,transmission_2,transmission_0,title_status_2,VIN_14,...,cylinders_3,cylinders_2,cylinders_1,cylinders_0,condition_2,condition_1,condition_0,model_9,model_8,model_7
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.0000,0.0000,...,1.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.0000,0.0000,...,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000
2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.0000,0.0000,...,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.0000,0.0000,...,1.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000
4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,...,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
5,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.0000,1.0000,...,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000
6,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.0000,1.0000,...,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
7,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.0000,1.0000,...,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
8,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,...,1.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000
9,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,1.0000,0.0000,...,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000


data_train_ft.info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200916 entries, 0 to 200915
Data columns (total 60 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   region_0        200916 non-null  float64
 1   title_status_0  200916 non-null  float64
 2   VIN_3           200916 non-null  float64
 3   VIN_2           200916 non-null  float64
 4   VIN_1           200916 non-null  float64
 5   VIN_0           200916 non-null  float64
 6   transmission_2  200916 non-null  float64
 7   transmission_0  200916 non-null  float64
 8   title_status_2  200916 non-null  float64
 9   VIN_14          200916 non-null  float64
 10  VIN_15          200916 non-null  float64
 11  VIN_16          200916 non-null  float64
 12  drive_0         200916 non-null  float64
 13  long            200916 non-null  float64
 14  lat             200916 non-null  float64
 15  state_5         200916 non-null  float64
 16  state_4         200916 non-null  f

None

### Step 5
Convert all feature values to floating-point numbers for better accuracy when performing tests later.

### Step 6
Create a library of different models to train and track the accuracy of later. Like the feature selection, having a wide variety of models to test out helps us ensure that our predictions are as accurate as possible.

In [ ]:
scaler = skpre.StandardScaler()

pipelines_to_try = {}

for k in range(36,45,2):
    pipelines_to_try[f'DTC-{k}'] =  tree.DecisionTreeClassifier(max_depth=k)
for k in range(70,120, 30):
    pipelines_to_try[f'RandomForestClassifier-{k}'] = RandomForestClassifier(n_estimators=k)
baseline = dummy.DummyClassifier(strategy="uniform")
pipelines_to_try1 = {}
# scale the data
for model_name in pipelines_to_try:
    pipelines_to_try1[f'std_{model_name}_pipe'] = pipeline.make_pipeline(scaler, 
                                                      pipelines_to_try[model_name])


### Step 8
Use cross-validation to test the accuracy of the models.

In [ ]:
accuracy_scores = {}
loo = skms.LeaveOneOut()
for name, model in pipelines_to_try1.items():
    loo = skms.LeaveOneOut()
    print(f'Currently modeling using : {name}')
    scores = skms.cross_val_score(model,
                                  data_train_ft,
                                  data_train_tgt,
                                  #cv=loo,
                                  #n_jobs=-1,
                                  cv=10,
                                  scoring='accuracy')
    mean_accuracy = scores.mean()
    accuracy_scores[name] = mean_accuracy
    print(f'{name}: {mean_accuracy:.3f}')

Currently modeling using : std_DTC-36_pipe
std_DTC-36_pipe: 0.710
Currently modeling using : std_DTC-38_pipe
std_DTC-38_pipe: 0.710
Currently modeling using : std_DTC-40_pipe
std_DTC-40_pipe: 0.710
Currently modeling using : std_DTC-42_pipe
std_DTC-42_pipe: 0.711
Currently modeling using : std_DTC-44_pipe
std_DTC-44_pipe: 0.710
Currently modeling using : std_RandomForestClassifier-70_pipe
std_RandomForestClassifier-70_pipe: 0.777
Currently modeling using : std_RandomForestClassifier-100_pipe


KeyboardInterrupt: 

### Step 9
Find the pipeline with the highest accuracy and save it for testing.

In [ ]:
best_pipeline_name = max(accuracy_scores,key=accuracy_scores.get)
print(f'\nBest pipeline: {best_pipeline_name} (accuracy = {accuracy_scores[best_pipeline_name]:.3f})')
final_pipeline = pipelines_to_try1[best_pipeline_name]

# Best model seems to be RandomForestClassifier (accuracy = 0.954)


Best pipeline: std_RandomForestClassifier-70_pipe (accuracy = 0.415)


### Step 10
Fit and test the highest-scoring model against the testing data. Save the results in submission.csv for uploading to Kaggle.

In [ ]:
scaler.fit(data_train_ft)

train_scaled_ftrs = scaler.transform(data_train_ft)

# transform validation features
test_scaled_ftrs = scaler.transform(data_test_df)

fit = final_pipeline.fit(train_scaled_ftrs, data_train_tgt)
predictions = fit.predict(test_scaled_ftrs)
#fit = final_pipeline.fit(data_train_ft, data_train_tgt)
#predictions = fit.predict(data_test_df)
make_submission_df = pd.read_csv("cars_class_test.csv")
# drop all columns except 'id'
make_submission_df = make_submission_df[['id']]
# make sure the column of ID's that we just read in is the index column
make_submission_df = make_submission_df.set_index('id')
# write to csv file
make_submission_df['type'] = predictions
make_submission_df.to_csv('submission.csv',sep=',')
# display message
print("Saved predictions to csv file.")

Saved predictions to csv file.
